In [1]:
import numpy as np
import cv2
import time
import numpy as np
import struct
import socket
import TcpUdpHelper
import PowerSensor as ps

# TCP UDP服务器

## 构造自定义的数据类

In [2]:
class PowersensorPara_ex(TcpUdpHelper.PowersensorPara):
    def __init__(self, debug=False):
        super(PowersensorPara_ex, self).__init__()
        self.paraSwitch = [False, False, False, False, False, False, False, False]
        self.paraInt = np.array(np.zeros(2), dtype=np.int32)
        self.paraFloat = np.array(np.zeros(8), dtype=np.float32)
    
    def unpackBuffer(self, data_buf):
        for i in range(8):
            # print(data_buf[4 + i] )
            if data_buf[4 + i] > 0:
                self.paraSwitch[i] = True
            else:
                self.paraSwitch[i] = False    
        temp = struct.unpack('ii', data_buf[12:20])
        self.paraInt = np.array(temp, np.int32)
        temp = struct.unpack('ffffffff', data_buf[20:52])
        self.paraFloat = np.array(temp, np.float32)
        
    def packParas(self):
        buffer = struct.pack("iiffffffff", self.paraInt[0], self.paraInt[1], self.paraFloat[0], self.paraFloat[1],
                            self.paraFloat[2], self.paraFloat[3], self.paraFloat[4], self.paraFloat[5],
                            self.paraFloat[6], self.paraFloat[7])
        cmd_buf = bytearray(52)
        cmd_buf[0] = 0xfa
        cmd_buf[1] = 0xf5
        cmd_buf[2] = 0
        cmd_buf[3] = 0x02
        for i in range(8):
            if self.paraSwitch[i]:
                cmd_buf[i + 4] = 1
            else:
                cmd_buf[i + 4] = 0
        for i in range(len(buffer)):
            cmd_buf[i + 12] = buffer[i]
        return cmd_buf
    
    


## 新建服务器

In [3]:
x = PowersensorPara_ex()
# 端口号必须和上位机定义的一致
server = TcpUdpHelper.PowersensorServer(port=10775, para=x)

## 启动服务器

In [4]:
x = server.startServer()

[Errno 104] Connection reset by peer
[Errno 104] Connection reset by peer


## 测试

In [12]:
# 打印原始数据 
print('开关型数据:' + str(x.paraSwitch))
print('整型数据:' + str(x.paraInt))
print('浮点型数据:' + str(x.paraFloat))

开关型数据:[True, False, False, True, True, False, True, False]
整型数据:[10 10]
浮点型数据:[ 0.1 -0.2  0.3 -0.4  0.5 -0.6 -0.7 -0.8]


In [13]:
# 在上位机上修改数据后，点写入参数，然后打印查看参数的变化
print('开关型数据:' + str(x.paraSwitch))
print('整型数据:' + str(x.paraInt))
print('浮点型数据:' + str(x.paraFloat))

开关型数据:[True, False, False, True, True, False, True, False]
整型数据:[34 10]
浮点型数据:[ 0.1 -0.2  0.3 -0.4  0.5 -0.6 -0.7 -0.8]


In [14]:
# 修改值后，在上位机上可以看到读书发生变化
x.paraFloat[3] = 0.555

## UDP图像显示

1. 通过循环把图像置入x.img既可在上位机上显示图像
2. 在上位机上点击显示图像

In [6]:
cam1 = ps.ImageSensor()

In [8]:
for i in range(100): 
    start = time.time() 
#     clear_output(wait=True) 
    imgMat = cam1.read_img_ori() 
    # 缩放图像，以减小保存的文件体积 
    tempImg = cv2.resize(imgMat, (320,240)) 
    x.img = tempImg
#     ps.CommonFunction.show_img_jupyter(tempImg) 
    time.sleep(0.1)